In [1]:
%reload_ext autoreload
%autoreload 2

# Use aligner_v7 kernel

import sys
add_paths = [
    "/fsx_0/user/tranx/rsync", # ALIGNER_PARENT_DIR
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/replicated", # ALIGNER_PARENT_DIR/llm_mm_aligner/replicated
    # "/data/home/tranx/conda/envs/aligner_20240822_v2/python-packages", #CONDA_PREFIX/python-packages
    # "/data/home/kapilk/.conda/envs/aligner_20240822_v2/python-packages"
    "/fsx_0/shared/conda/aligner_20241030/python-packages"
]

for p in add_paths:
    if p not in sys.path:
        sys.path.append(p)
        
device = "cuda:2"

In [2]:
import torch

In [3]:
MODEL_H = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
MODEL_G = "/fsx_0/user/marcomonteiro/experiments/G/VEv0p2_G14_aws_final_256x8x98_next/checkpoints/epoch_450.pt"

In [4]:
sd_h = torch.load(MODEL_H, map_location="cpu")

In [5]:
sd_g = torch.load(MODEL_G, map_location="cpu")

: 

In [2]:
import torch
from llm_mm_aligner.lib.encoders.collator import PACKED
from llm_mm_aligner.lib.encoders.metaclip_fixed_resolution import MetaCLIP
# from llm_mm_aligner.lib.encoders.metaclip_native_resolution import MetaCLIP_NAVIT
from llm_mm_aligner.lib.encoders.metaclip_navit import MetaCLIP_NAVIT
from PIL import Image
from torchvision import transforms

/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aligner_v7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_CHECKPOINT = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
TEST_IMAGE = "/fsx_0/dataset01/coco/val2014/COCO_val2014_000000485307.jpg"

metaclip_config = {
    "ckpt_path": MODEL_CHECKPOINT,
    "image_size": 336,  # image_size can also be 448, since the position embedding is rope_2d
    "patch_size": 14,
    "width": 1408,
    "mlp_ratio": 4.0,
    "layers": 34,
    "heads": 16,
    "load_ckpt": True,
    "relative_pos_embed_type": "rope_2d",
}

CLIP_MEAN = (0.4814546, 0.4578275, 0.40821073)
CLIP_STD = (0.2686295, 0.2613025, 0.2757711)

In [4]:
base_model =  MetaCLIP(**metaclip_config)
base_model.cuda().eval()
print("Loaded MetaCLIP fixed resolution model.")

Unexpected keys: ['positional_embedding']
Loaded MetaCLIP fixed resolution model.


In [5]:
base_model

MetaCLIP(
  (conv1): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-33): 34 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionXformer4x(
          (out_proj): Linear(in_features=1408, out_features=1408, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1408, out_features=5632, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=5632, out_features=1408, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
)

In [6]:
navit_model = MetaCLIP_NAVIT(**metaclip_config)
navit_model.cuda().eval()
print("Loaded MetaCLIP native resolution model.")

Unexpected keys: []
Loaded MetaCLIP native resolution model.


In [7]:
def preprocess_image(image_path):
    preprocess = transforms.Compose(
        [
            transforms.Resize(metaclip_config["image_size"]),
            transforms.CenterCrop(metaclip_config["image_size"]),
            transforms.ToTensor(),
            transforms.Normalize(mean=CLIP_MEAN, std=CLIP_STD),
        ]
    )

    image = Image.open(image_path).convert("RGB")
    return preprocess(image).unsqueeze(0)

def pack_window(img):
    patch_h = patch_w = metaclip_config["patch_size"]
    _, channel, image_h, image_w = img.shape
    img = img.unfold(-2, patch_h, patch_h).unfold(-2, patch_w, patch_w)

    idx_h, idx_w = image_h // patch_h, image_w // patch_w
    img_idx = torch.arange(
        image_h * image_w // (patch_h * patch_w), dtype=torch.int32
    )
    img_idx = img_idx.reshape(idx_h * idx_w, 1)

    img = img.reshape(channel, -1, 1, patch_h, patch_w)
    img_idx = img_idx.reshape(-1, 1)

    img = torch.cat([img, img[:, :1]], dim=1)
    img_idx = torch.cat([img_idx, img_idx[:1]], dim=0)
    img_idx[-1, -1] = PACKED.ID_CLS_TOKEN

    packed_img_idx = torch.empty(
        img_idx.shape[0],
        img_idx.shape[1],
        PACKED.NUM_METADATA - 1,
        dtype=torch.int32,
    )

    packed_img_idx[:, :, PACKED.Y] = img_idx // idx_w
    packed_img_idx[:, :, PACKED.X] = img_idx % idx_w
    packed_img_idx[:, :, PACKED.IDX] = img_idx

    return img, packed_img_idx

In [9]:
# test forward parity
image_tensor_base = (
    preprocess_image(TEST_IMAGE).cuda().repeat(8, 1, 1, 1)
)  # [8, 3, 336, 336]
print("image_tensor_base:", image_tensor_base.shape)
with torch.no_grad():
    base_features = base_model(image_tensor_base)
print("base_features.shape:", base_features.shape)
# print(base_features)

image_tensor_navit = preprocess_image(TEST_IMAGE).cuda().repeat(8, 1, 1, 1)
print("image_tensor_navit:", image_tensor_navit.shape)
batch_size = image_tensor_navit.shape[0]

images = []
packed_img_idxs = []

for i in range(batch_size):
    print(i)
    image = image_tensor_navit[i].unsqueeze(0)  # [8, 3, 336, 336]
    print(f"image.shape={image.shape}")
    packed_img, packed_img_idx_i = pack_window(image)
    print(f"packed_img.shape={packed_img.shape}")
    images.append(packed_img)
    packed_img_idxs.append(packed_img_idx_i)
    
tokens_per_img = torch.tensor(
    [packed_img.shape[1] for packed_img in images]
).long()

packed_img_idxs = torch.cat(packed_img_idxs, dim=0)
images = torch.cat(images, dim=1).permute(1, 0, 2, 3, 4).contiguous()
images_package = [images.cuda(), packed_img_idxs.cuda(), tokens_per_img.cuda()]

image_tensor_base: torch.Size([8, 3, 336, 336])
base_features.shape: torch.Size([8, 576, 1408])
image_tensor_navit: torch.Size([8, 3, 336, 336])
0
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
1
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
2
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
3
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
4
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
5
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
6
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])
7
image.shape=torch.Size([1, 3, 336, 336])
packed_img.shape=torch.Size([3, 577, 1, 14, 14])


In [12]:
images_package.shape

AttributeError: 'list' object has no attribute 'shape'

In [13]:
with torch.no_grad():
    navit_features = torch.stack(navit_model(image_tensor_navit), dim=0)

/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_navit.py:287: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor